In [19]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set your executable path for scraping
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Visit the mars nasa news site
#assign the url and instruct the browser to visit it.
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
# One is that we're searching for elements with a specific combination of tag (div) and attribute (list_text). 
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [ ]:
###  set up the HTML parser:
##  html = browser.html
### Assigned parent element "slide_elem" as the variable to look for the <div /> tag and its descendent.
###    news_soup = soup(html, 'html.parser')
### the period in div.list_text pinpoints the div/ tag with the class of List_text.  Css works from right to left
#   slide_elem = news_soup.select_one('div.list_text')

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
#assign the title and summary text to variables
slide_elem.find('div', class_='content_title')

<div class="content_title">The MarCO Mission Comes to an End</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title')
news_title

<div class="content_title">The MarCO Mission Comes to an End</div>

In [7]:
# Use the parent element to find the first `a` tag and save it as `news_title`
#Searching within parent element slide_elem for the title and stripping additonal HTML attributes .get_text()
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'The MarCO Mission Comes to an End'

In [8]:
# add the summary text
news_title = slide_elem.find('div', class_='article_teaser_body').get_text()
news_title

'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'

In [9]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'

# Scrape the featured image from another Mars website

In [13]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [14]:
# Find and click the full image button.  The browsser finds an element by its tag
# full_image_elem is a new variable to hold the scraping results
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [15]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [16]:
# Find the relative image url
#note that the value of the src will be different every time the page is updated
# An img tag is nested within this HTML, so we've included it & .get('src') pulls the link to the image.
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel


'image/featured/mars2.jpg'

In [17]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Fact scraping from tables 

In [ ]:
# Visit URL url = 'https://galaxyfacts-mars.com/'

In [ ]:
# Tables in HTML are basically made up of many smaller containers.
# The main container is the <table /> tag. Inside the table is <tbody />, 
# which is the body of the table—the headers, columns, and rows.
# <tr /> is the tag for each table row. Within that tag, the table data is stored in <td /> tags. 
# This is where the columns are established.
# Instead of scraping each row use Pandas' .read_html() function to scrape the entire table via DataFrame

In [20]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [21]:
# convert our DataFrame back into HTML-ready code using the .to_html() function.
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [22]:
browser.quit()